In [1]:
import numpy as np
import cv2

————————————————————————————————————————————————————

实现图像拼接步骤

1. 采用sift特征检测算法检测两幅图像的关键特征点

2. 采用knn检测函数进行特征匹配并可视化

3. 从所匹配的全部关键点中筛选出优秀的特征点（基于距离筛选）

4. 用 RANSAC 算法来计算透视变换矩阵 H

5. 使用透视变换矩阵H对图片1/2进行透视变换，得到变换后的结果图像

6. 将图片2/1叠加在变换后的图像result的左上角位置，实现图像融合

7. 图像融合后会出现黑色边界区域，通过中值滤波去除黑色区域

In [5]:

# ################################
# 使用 OpenCV 进行 SIFT 特征提取
# ################################
def Get_features(gray_image):

    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)
    
    return keypoints, descriptors

# ################################
#           获取图片
# ################################
def Get_img(image_path):
    image =  cv2.imread(image_path)
    #OpenCV中读取的图像是以BGR（蓝绿红）顺序进行编码的，需要将图像转换为RGB格式进行处理
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #防止图片过大
    img = cv2.resize(gray_image, (1028, 762))
    return img

# ################################
# 采用knn检测函数进行特征匹配
# 获取匹配的关键点用以可视化
# ################################
def Get_match(feature_1,feature_2):
    bf = cv2.BFMatcher()
    rawMatches = bf.knnMatch(feature_1, feature_2, k=2)

    matches = []
    #过滤
    for m,n in rawMatches:
            if m.distance < 0.75 * n.distance:
                matches.append(m)
                        
    return  matches


# ################################
#         显示图片并保存
# ################################
def show_img(name,img):
    #获取图片时转换为RGB格式，显示图片则需要转换为BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imshow(name,img)
    cv2.imwrite('img/'+ name + '.jpg', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

# #################################################################
# 采用knn检测函数进行特征匹配
# 用 RANSAC 算法来计算透视变换矩阵 H，并返回变换矩阵、匹配特征点对和状态信息
# #################################################################

def matchKeypoints(keypoints_1, keypoints_2, features_1, features_2, ratio, reprojThresh):
        # 创建BFMatcher对象
        matcher = cv2.BFMatcher()
        # 使用BFMatcher进行特征点匹配
        rawMatches = matcher.knnMatch(features_1, features_2, 2)
        
        #受cv2.findHomography()函数输入限制，更改类型
        keypoints_1 = np.float32([kp.pt for kp in keypoints_1])
        keypoints_2 = np.float32([kp.pt for kp in keypoints_2])
        
        matches = []
        for m in rawMatches:
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))

       #如果筛选后的匹配特征点对数量大于 4，则将这些特征点对转换为 NumPy 数组
        if len(matches) > 4:
            ptsA = np.float32([keypoints_1[i] for (_, i) in matches])
            ptsB = np.float32([keypoints_2[i] for (i, _) in matches])
            
            #用 RANSAC 算法来计算透视变换矩阵 H，并返回变换矩阵、匹配特征点对和状态信息。
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
            
            return (matches, H, status)
        
        else:
            print("可匹配的特征点数量较低")

# ################################       
# 裁剪掉拼接后图像中的黑色区域  
# ################################

def remove_black_edge(image):
    # 使用中值滤波器对输入的图像进行中值滤波，目的是去除可能存在于黑色边缘中的噪声干扰
    img = cv2.medianBlur(image, 5)
    
    #使用阈值处理函数对中值滤波后的图像进行二值化处理。将像素值低于阈值15的像素设为0（黑色），将像素值高于阈值的像素设为255（白色）
    b = cv2.threshold(img, 15, 255, cv2.THRESH_BINARY)
    
    #b为阈值处理后的图像以及阈值处理的返回值。而我们只需要处理后的图像，因此取返回值的第二个元素b[1]
    binary_image = b[1]
    binary_image = cv2.cvtColor(binary_image, cv2.COLOR_BGR2GRAY)
    
    #获取原始图像的长和宽
    x = binary_image.shape[0]
    y = binary_image.shape[1]
    edges_x = []
    edges_y = []
    for i in range(x):
        for j in range(y):
            if binary_image[i][j] == 255:
                edges_x.append(i)
                edges_y.append(j)

    #重新设置图像的四点位置
    left = min(edges_x)
    right = max(edges_x)
    width = right - left
    bottom = min(edges_y)
    top = max(edges_y)
    height = top - bottom

    pre1_picture = image[left:left + width, bottom:bottom + height]
    return pre1_picture



In [7]:
if __name__ == "__main__":
    
    image_path_1 = 'img/1.jpg'
    image_path_2 = 'img/2.jpg'

    image_1 = Get_img(image_path_1)
    image_2 = Get_img(image_path_2)

    
    keypoints_1, descriptors_1 = Get_features(image_1)
    keypoints_2, descriptors_2 = Get_features(image_2)

    image_with_keypoints_1 = cv2.drawKeypoints(image_1, keypoints_1, None)
    image_with_keypoints_2 = cv2.drawKeypoints(image_2, keypoints_2, None)
    show_img('Image with Keypoints_1',image_with_keypoints_1)
    show_img('Image with Keypoints_2',image_with_keypoints_2)
    
    #显示图片的关键点
    match = Get_match(descriptors_1,descriptors_2)
    #绘制K近邻匹配结果
    image_with_match = cv2.drawMatches(image_1, keypoints_1, image_2, keypoints_2,match,None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    show_img('Image with matches',image_with_match)
    
   
    M = matchKeypoints(keypoints_2, keypoints_1, descriptors_2, descriptors_1, ratio = 0.75, reprojThresh = 4.0)
    if M is None:
        print("Error!")
    (matches, H, status) = M
    
    #使用透视变换矩阵H对image_1进行透视变换，得到变换后的结果图像result
    result = cv2.warpPerspective(image_2, H, ((image_1.shape[1] + image_2.shape[1])*2, max(image_1.shape[0],image_2.shape[0])))
    
    result_1 = remove_black_edge(result)
    #由于无法确定应当对哪张图片进行透视变换，因此判断 
    if np.size(result_1) <  np.size(image_2)*0.85:
        #交换图片
        image = image_1
        image_1 = image_2
        image_1 = image
        M = matchKeypoints(keypoints_1, keypoints_2, descriptors_1, descriptors_2, ratio = 0.75, reprojThresh = 4.0)
        (matches, H, status) = M
        #使用透视变换矩阵H对image_1进行透视变换，得到变换后的结果图像result
        result = cv2.warpPerspective(image_2, H, (image_1.shape[1] + image_2.shape[1], max(image_1.shape[0],image_2.shape[0])))
        
    show_img('images after perspective',result)

    #将image_2与变换后的图像result实现图像融合
    result[0:image_1.shape[0], 0:image_1.shape[1]] = image_1
#     result[0:image_1.shape[0], 0:image_1.shape[1]] = np.maximum(image_1, result[0:image_1.shape[0], 0:image_1.shape[1]])
    show_img('result images',result)
    
    
    result_2 = remove_black_edge(result)
    show_img('result images after remove black edge',result_2)

    